# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

from livelossplot import PlotLosses

# User Settings

In [2]:
import pickle

# class Company(object):
#     def __init__(self, name, value):
#         self.name = name
#         self.value = value

# with open('company_data.pkl', 'wb') as output:
#     company1 = Company('banana', 40)
#     pickle.dump(company1, output, pickle.HIGHEST_PROTOCOL)

#     company2 = Company('spam', 42)
#     pickle.dump(company2, output, pickle.HIGHEST_PROTOCOL)

# del company1
# del company2

# with open('company_data.pkl', 'rb') as input:
#     company1 = pickle.load(input)
#     print(company1.name)  # -> banana
#     print(company1.value)  # -> 40

#     company2 = pickle.load(input)
#     print(company2.name) # -> spam
#     print(company2.value)  # -> 42

    
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

# sample usage
#save_object(company1, 'company1.pkl')

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = False
save_masks = False
print_loss = True
output_tensors = False
train_flag = True
EPOCHS = 3
iteration = "-8"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 5
num_batch = 2000 #25000 #11112
val_batch = 20
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    best_model = None
    best_loss = math.inf
    #model.train()  
    #liveloss = PlotLosses()
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
        
        #Training    
        model.train()
        phase = 'train'
        epoch_loss = 0
        for batch in range(num_batch):
            running_loss = 0.0
            
            if print_loss:
                #print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                #dtype=torch.int64
                ### Why are we using float?
                #labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize, dtype = torch.float32).to(device)
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                #print("loading tensor:", i,  loc, flush=True)
                #print("tensor max:", torch.max(labels[i]))
                i += 1

            if train_flag:
                phase = 'train'
                #print("Setting model in training mode")
                # Set model to training mode
                model.train() 
            else:
                phase = 'val'
                #print("Setting model in validation mode")
                # Set model to evaluate mode
                model.eval()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                #outputs = model(inputs.to(device)).to(device)
                #outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            #
            
            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)
            #labels.to(device)

            # Moved outside paths forloop as labels would be reset
            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                log_name = str(datetime.now().date()) + iteration
                batch_log.write(log_dir, log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
                
        val_loss /= num_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)    

#             if show_live_loss and phase == 'train':
#                 logs['log loss'] = epoch_loss
#                 logs['accuracy'] = 1 - epoch_loss
#                 liveloss.update(logs)
#                 liveloss.draw()

#             if save_masks and phase == 'val':
#                 outputs = sigmoid_function(outputs)
#                 # need to fix before running
#                 outputs_dir = "data/outputs/results/" + str(datetime.now().date())
#                 os.makedirs(outputs_dir, exist_ok=True)
#                 threshold = 0.05
#                 j = 0
#                 for locations in paths:
#                     img = Image.open(locations)
#                     #num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
#                     for threshold in [0.05, .25, .5, .75]:
#                         img = data_factory.showMaskOnImage(img,outputs[j], threshold)
#                         img.save(outputs_dir  + "/val-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
#                     j += 1

            
                #input("please enter any key to continue")

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
            #torch.save(model, SAVE_LOCATION)
            #torch.save(model, SAVE_LOCATION.replace("model", "model_best"))
        
    time_elapsed = time.time() - since
    
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    log_name = str(datetime.now().date()) + iteration
    batch_log.write(log_dir, log_name)
    
    epoch_log.write(log_dir, "epoch - "+ str(datetime.now().date()) + iteration)
    
    #logs_dir = "data/logs/" + str(datetime.now().date())
    #save_object(log, logs_dir)
    
    #completed model
    torch.save(model,SAVE_LOCATION)
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
    print('initializing model with random weights', flush=True)
    torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
    
    torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
    
    #torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)

    #torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)

    torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)

    torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.05, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  


#model = model.to(device)

loading weights from ./data/models/model_best


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/3
Batch 1/2000
loss: 0.3354097306728363
---------------
Batch 2/2000
loss: 0.4637281894683838
---------------
Batch 3/2000
loss: 0.33553674817085266
---------------
Batch 4/2000
loss: 0.4824010729789734
---------------
Batch 5/2000
loss: 0.38421615958213806
---------------
Batch 6/2000
loss: 0.3491530418395996
---------------
Batch 7/2000
loss: 0.28928342461586
---------------
Batch 8/2000
loss: 0.42487430572509766
---------------
Batch 9/2000
loss: 0.3685886561870575
---------------
Batch 10/2000
loss: 0.3142160475254059
---------------
Batch 11/2000
loss: 0.4480555057525635
---------------
Batch 12/2000
loss: 0.2550874352455139
---------------
Batch 13/2000
loss: 0.42420053482055664
---------------
Batch 14/2000
loss: 0.3269560933113098
---------------
Batch 15/2000
loss: 0.311749130487442
---------------
Batch 16/2000
loss: 0.3206559121608734
---------------
Batch 17/2000
loss: 0.5554056763648987
---------------
Batch 18/2000
loss: 0.2694284915924072
---------------
Batch 19

loss: 0.17784994840621948
---------------
Batch 149/2000
loss: 0.28419509530067444
---------------
Batch 150/2000
loss: 0.4017910361289978
---------------
Batch 151/2000
loss: 0.392269492149353
---------------
Batch 152/2000
loss: 0.27335309982299805
---------------
Batch 153/2000
loss: 0.4361424148082733
---------------
Batch 154/2000
loss: 0.3244476914405823
---------------
Batch 155/2000
loss: 0.325459748506546
---------------
Batch 156/2000
loss: 0.3867442011833191
---------------
Batch 157/2000
loss: 0.38647207617759705
---------------
Batch 158/2000
loss: 0.3656838536262512
---------------
Batch 159/2000
loss: 0.38639312982559204
---------------
Batch 160/2000
loss: 0.5419514179229736
---------------
Batch 161/2000
loss: 0.36594441533088684
---------------
Batch 162/2000
loss: 0.3599323332309723
---------------
Batch 163/2000
loss: 0.2988380789756775
---------------
Batch 164/2000
loss: 0.19628891348838806
---------------
Batch 165/2000
loss: 0.21127727627754211
---------------
B

---------------
Batch 294/2000
loss: 0.31622472405433655
---------------
Batch 295/2000
loss: 0.33436423540115356
---------------
Batch 296/2000
loss: 0.3174470365047455
---------------
Batch 297/2000
loss: 0.42036256194114685
---------------
Batch 298/2000
loss: 0.33039969205856323
---------------
Batch 299/2000
loss: 0.32964444160461426
---------------
Batch 300/2000
loss: 0.280236154794693
---------------
Batch 301/2000
loss: 0.294108510017395
---------------
Batch 302/2000
loss: 0.2712908983230591
---------------
Batch 303/2000
loss: 0.28485265374183655
---------------
Batch 304/2000
loss: 0.33884355425834656
---------------
Batch 305/2000
loss: 0.3145522177219391
---------------
Batch 306/2000
loss: 0.3426111340522766
---------------
Batch 307/2000
loss: 0.551397979259491
---------------
Batch 308/2000
loss: 0.32470402121543884
---------------
Batch 309/2000
loss: 0.3309095799922943
---------------
Batch 310/2000
loss: 0.2319565862417221
---------------
Batch 311/2000
loss: 0.2800

Batch 439/2000
loss: 0.3491601347923279
---------------
Batch 440/2000
loss: 0.3091089129447937
---------------
Batch 441/2000
loss: 0.35634586215019226
---------------
Batch 442/2000
loss: 0.33953922986984253
---------------
Batch 443/2000
loss: 0.3534491956233978
---------------
Batch 444/2000
loss: 0.37475427985191345
---------------
Batch 445/2000
loss: 0.22077339887619019
---------------
Batch 446/2000
loss: 0.3109833598136902
---------------
Batch 447/2000
loss: 0.33268386125564575
---------------
Batch 448/2000
loss: 0.28540363907814026
---------------
Batch 449/2000
loss: 0.3628470003604889
---------------
Batch 450/2000
loss: 0.33217790722846985
---------------
Batch 451/2000
loss: 0.27216026186943054
---------------
Batch 452/2000
loss: 0.28125110268592834
---------------
Batch 453/2000
loss: 0.28158649802207947
---------------
Batch 454/2000
loss: 0.34077388048171997
---------------
Batch 455/2000
loss: 0.2641059160232544
---------------
Batch 456/2000
loss: 0.30223652720451

---------------
Batch 585/2000
loss: 0.32545843720436096
---------------
Batch 586/2000
loss: 0.31318598985671997
---------------
Batch 587/2000
loss: 0.2634679973125458
---------------
Batch 588/2000
loss: 0.29265034198760986
---------------
Batch 589/2000
loss: 0.2881411612033844
---------------
Batch 590/2000
loss: 0.27695760130882263
---------------
Batch 591/2000
loss: 0.37081342935562134
---------------
Batch 592/2000
loss: 0.18975615501403809
---------------
Batch 593/2000
loss: 0.3042556047439575
---------------
Batch 594/2000
loss: 0.2847405672073364
---------------
Batch 595/2000
loss: 0.20358334481716156
---------------
Batch 596/2000
loss: 0.37041595578193665
---------------
Batch 597/2000
loss: 0.26747363805770874
---------------
Batch 598/2000
loss: 0.34927132725715637
---------------
Batch 599/2000
loss: 0.3647103011608124
---------------
Batch 600/2000
loss: 0.3713071942329407
---------------
Batch 601/2000
loss: 0.4115343391895294
---------------
Batch 602/2000
loss: 0

...

Batch 1449/2000
loss: 0.2937205135822296
---------------
Batch 1450/2000
loss: 0.33732372522354126
---------------
Batch 1451/2000
loss: 0.22356592118740082
---------------
Batch 1452/2000
loss: 0.3584721088409424
---------------
Batch 1453/2000
loss: 0.16468967497348785
---------------
Batch 1454/2000
loss: 0.32205939292907715
---------------
Batch 1455/2000
loss: 0.25132930278778076
---------------
Batch 1456/2000
loss: 0.3372984826564789
---------------
Batch 1457/2000
loss: 0.34825003147125244
---------------
Batch 1458/2000
loss: 0.2855457663536072
---------------
Batch 1459/2000
loss: 0.37676963210105896
---------------
Batch 1460/2000
loss: 0.23640401661396027
---------------
Batch 1461/2000
loss: 0.19788388907909393
---------------
Batch 1462/2000
loss: 0.25589385628700256
---------------
Batch 1463/2000
loss: 0.3669755756855011
---------------
Batch 1464/2000
loss: 0.27183598279953003
---------------
Batch 1465/2000
loss: 0.3679110109806061
---------------
Batch 1466/2000
loss

---------------
Batch 1592/2000
loss: 0.3755127191543579
---------------
Batch 1593/2000
loss: 0.30169010162353516
---------------
Batch 1594/2000
loss: 0.3154148757457733
---------------
Batch 1595/2000
loss: 0.3532085716724396
---------------
Batch 1596/2000
loss: 0.37776997685432434
---------------
Batch 1597/2000
loss: 0.35895657539367676
---------------
Batch 1598/2000
loss: 0.3184419274330139
---------------
Batch 1599/2000
loss: 0.22082842886447906
---------------
Batch 1600/2000
loss: 0.4559498429298401
---------------
Batch 1601/2000
loss: 0.3326267600059509
---------------
Batch 1602/2000
loss: 0.33181232213974
---------------
Batch 1603/2000
loss: 0.34809014201164246
---------------
Batch 1604/2000
loss: 0.3355817496776581
---------------
Batch 1605/2000
loss: 0.3602864742279053
---------------
Batch 1606/2000
loss: 0.42200833559036255
---------------
Batch 1607/2000
loss: 0.3159129023551941
---------------
Batch 1608/2000
loss: 0.27267947793006897
---------------
Batch 1609

---------------
Batch 1735/2000
loss: 0.4307326078414917
---------------
Batch 1736/2000
loss: 0.29444050788879395
---------------
Batch 1737/2000
loss: 0.365560919046402
---------------
Batch 1738/2000
loss: 0.35151344537734985
---------------
Batch 1739/2000
loss: 0.31187254190444946
---------------
Batch 1740/2000
loss: 0.23105600476264954
---------------
Batch 1741/2000
loss: 0.28828656673431396
---------------
Batch 1742/2000
loss: 0.39709407091140747
---------------
Batch 1743/2000
loss: 0.4045894145965576
---------------
Batch 1744/2000
loss: 0.30620506405830383
---------------
Batch 1745/2000
loss: 0.311419278383255
---------------
Batch 1746/2000
loss: 0.21629022061824799
---------------
Batch 1747/2000
loss: 0.32310476899147034
---------------
Batch 1748/2000
loss: 0.24733886122703552
---------------
Batch 1749/2000
loss: 0.22816205024719238
---------------
Batch 1750/2000
loss: 0.31057241559028625
---------------
Batch 1751/2000
loss: 0.32383885979652405
---------------
Batc

loss: 0.3717648684978485
---------------
Batch 1878/2000
loss: 0.2836327850818634
---------------
Batch 1879/2000
loss: 0.22920158505439758
---------------
Batch 1880/2000
loss: 0.4135890603065491
---------------
Batch 1881/2000
loss: 0.2780183255672455
---------------
Batch 1882/2000
loss: 0.40030935406684875
---------------
Batch 1883/2000
loss: 0.24251066148281097
---------------
Batch 1884/2000
loss: 0.38527044653892517
---------------
Batch 1885/2000
loss: 0.366056352853775
---------------
Batch 1886/2000
loss: 0.28414520621299744
---------------
Batch 1887/2000
loss: 0.279496967792511
---------------
Batch 1888/2000
loss: 0.37655603885650635
---------------
Batch 1889/2000
loss: 0.36686429381370544
---------------
Batch 1890/2000
loss: 0.33745214343070984
---------------
Batch 1891/2000
loss: 0.2821001708507538
---------------
Batch 1892/2000
loss: 0.3391798138618469
---------------
Batch 1893/2000
loss: 0.30614185333251953
---------------
Batch 1894/2000
loss: 0.3422594368457794

loss: 0.2960089445114136
---------------
Training complete in 490.0m 24.3s


# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [11]:
dat = dataGenerator.dataGenerator(416)
j = 0
outputs_dir = "data/outputs/" + str(datetime.now().date())
os.makedirs(outputs_dir, exist_ok=True)
threshold = 0.05
batch = 3

for locations in paths:
    img = Image.open(locations)
    img = dat.showMaskOnImage(img,outputs[j], threshold)
    img.save(outputs_dir  + "/batch_" + str(batch) + "-" + str(j) + ".png","PNG")
    #img.show()
    j += 1

#Cells
outputs[0]
    
#Cells
locations = "/data/photos/999.png"
loc = locations.replace("/data/photos/", "").replace(".png", "")

print(locations)
print(loc)

In [8]:
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)

In [3]:
model = nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)
input = torch.rand(1, 3, 216, 216)

In [4]:
input.size()

torch.Size([1, 3, 216, 216])

In [5]:
output = model(input)

In [6]:
output.size()

torch.Size([1, 64, 108, 108])

In [4]:
torchvision.models.vgg16(pretrained=True).features

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (17): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(